In [1]:
import mysql.connector
import boto3
from boto3.dynamodb.conditions import Key, Attr
from datetime import datetime
from decimal import Decimal
import csv
import pprint
import requests
import json

In [2]:
#periodo de analise dos cpfs
inicio = datetime.strptime('2021-07-09 00:00:00', '%Y-%m-%d %H:%M:%S')
fim = datetime.strptime('2021-07-09 23:59:59', '%Y-%m-%d %H:%M:%S')
cpfs = {}

In [3]:
#conectando ao dynamo
dynamodb = boto3.resource('dynamodb',
                          region_name='us-east-1',
                          aws_access_key_id='#######',
                          aws_secret_access_key='######')

In [4]:
#tabela para pegar os dados do serasa
table = dynamodb.Table('evaluation-report-data-prod')

#tabela para salvar os da2dos do boa vista e quod
table2 = dynamodb.Table('experimento-comparacao-prod')

In [5]:
f = Attr('applicant_id').ne(0)
response = table.scan(FilterExpression=f)

while 'LastEvaluatedKey' in response:
    for i in response['Items']:
        data_analise = datetime.strptime(i['data']['laudo']['data_conclusao'], '%Y-%m-%d %H:%M:%S')
        if data_analise >= inicio and data_analise <= fim:
            print('CPF: ', i['data']['pretendente']['cpf'])
            print('Consultas anteriores: ', i['data']['laudo']['rede_ficha']['pesquisas_anteriores_api']['result']['total'])
            print('Pendências: ', len(i['data']['laudo']['restricoes_financeiras']['result']['pendencias']['detalhes']))
            print('Protestos: ' , len(i['data']['laudo']['restricoes_financeiras']['result']['protestos']['detalhes']))
            print('Cheques sem fundo: ', len(i['data']['laudo']['restricoes_financeiras']['result']['cheques']['detalhes']))
            print('------------')
    if 'LastEvaluatedKey' in response:
        response = table.scan(FilterExpression=f, ExclusiveStartKey=response['LastEvaluatedKey'])

CPF:  12327064775
Consultas anteriores:  1
Pendências:  0
Protestos:  0
Cheques sem fundo:  0
------------
CPF:  11653676884
Consultas anteriores:  0
Pendências:  2
Protestos:  0
Cheques sem fundo:  0
------------
CPF:  29859891915
Consultas anteriores:  0
Pendências:  0
Protestos:  0
Cheques sem fundo:  0
------------
CPF:  22370609915
Consultas anteriores:  0
Pendências:  0
Protestos:  0
Cheques sem fundo:  0
------------
CPF:  41603931953
Consultas anteriores:  0
Pendências:  0
Protestos:  0
Cheques sem fundo:  0
------------
CPF:  45527997972
Consultas anteriores:  1
Pendências:  0
Protestos:  0
Cheques sem fundo:  0
------------
CPF:  46557202049
Consultas anteriores:  1
Pendências:  1
Protestos:  0
Cheques sem fundo:  0
------------
CPF:  95269991004
Consultas anteriores:  1
Pendências:  0
Protestos:  0
Cheques sem fundo:  0
------------
CPF:  93681151091
Consultas anteriores:  0
Pendências:  0
Protestos:  0
Cheques sem fundo:  0
------------
CPF:  10100913954
Consultas anteriore

CPF:  86764330468
Consultas anteriores:  0
Pendências:  1
Protestos:  0
Cheques sem fundo:  0
------------
CPF:  02809634750
Consultas anteriores:  0
Pendências:  2
Protestos:  0
Cheques sem fundo:  0
------------
CPF:  15073415710
Consultas anteriores:  1
Pendências:  1
Protestos:  0
Cheques sem fundo:  0
------------
CPF:  03508482979
Consultas anteriores:  0
Pendências:  1
Protestos:  0
Cheques sem fundo:  0
------------
CPF:  09702759781
Consultas anteriores:  0
Pendências:  0
Protestos:  0
Cheques sem fundo:  0
------------
CPF:  81581157053
Consultas anteriores:  1
Pendências:  0
Protestos:  0
Cheques sem fundo:  0
------------
CPF:  03129410058
Consultas anteriores:  0
Pendências:  0
Protestos:  0
Cheques sem fundo:  0
------------
CPF:  01135476250
Consultas anteriores:  0
Pendências:  0
Protestos:  0
Cheques sem fundo:  0
------------
CPF:  16089679720
Consultas anteriores:  0
Pendências:  0
Protestos:  0
Cheques sem fundo:  0
------------
CPF:  03040004000
Consultas anteriore

CPF:  14937431714
Consultas anteriores:  0
Pendências:  0
Protestos:  0
Cheques sem fundo:  0
------------
CPF:  10837980909
Consultas anteriores:  0
Pendências:  0
Protestos:  0
Cheques sem fundo:  0
------------
CPF:  02456125991
Consultas anteriores:  0
Pendências:  0
Protestos:  0
Cheques sem fundo:  0
------------
CPF:  94020167915
Consultas anteriores:  1
Pendências:  0
Protestos:  1
Cheques sem fundo:  0
------------
CPF:  10594024757
Consultas anteriores:  0
Pendências:  0
Protestos:  0
Cheques sem fundo:  0
------------
CPF:  05735128760
Consultas anteriores:  0
Pendências:  0
Protestos:  0
Cheques sem fundo:  0
------------
CPF:  66307120720
Consultas anteriores:  0
Pendências:  0
Protestos:  0
Cheques sem fundo:  0
------------
CPF:  26289768700
Consultas anteriores:  2
Pendências:  0
Protestos:  0
Cheques sem fundo:  0
------------
CPF:  02234276918
Consultas anteriores:  0
Pendências:  0
Protestos:  0
Cheques sem fundo:  0
------------
CPF:  09635971915
Consultas anteriore

In [9]:
#SERASA - pegando os cpfs ja existentes em nossa base
f = Attr('applicant_id').ne(0)
response = table.scan(FilterExpression=f)

while 'LastEvaluatedKey' in response and len(cpfs) < 42:
    for i in response['Items']:
        data_analise = datetime.strptime(i['data']['laudo']['data_conclusao'], '%Y-%m-%d %H:%M:%S')
        if data_analise >= inicio and data_analise <= fim:
            cpf = i['data']['pretendente']['cpf']
            table2.put_item(
               Item={
                    'document': cpf,
                    'bureau': 'serasa',
                    'data': i['data']
                    }
            )
            if cpf not in cpfs and cpf not in cpfs_ja_capturados:
                cpfs.append(cpf)
                print('CPF ADICIONADO NA LISTA: ', cpf)
                print('Total de CPFs: ', len(cpfs))
                print('----')
    if 'LastEvaluatedKey' in response:
        response = table.scan(FilterExpression=f, ExclusiveStartKey=response['LastEvaluatedKey'])

CPF ADICIONADO NA LISTA:  40485160862
Total de CPFs:  1
----
CPF ADICIONADO NA LISTA:  01179663713
Total de CPFs:  2
----
CPF ADICIONADO NA LISTA:  22086099720
Total de CPFs:  3
----
CPF ADICIONADO NA LISTA:  07598618704
Total de CPFs:  4
----
CPF ADICIONADO NA LISTA:  41587278987
Total de CPFs:  5
----
CPF ADICIONADO NA LISTA:  59116587987
Total de CPFs:  6
----
CPF ADICIONADO NA LISTA:  01110481098
Total de CPFs:  7
----
CPF ADICIONADO NA LISTA:  01317331176
Total de CPFs:  8
----
CPF ADICIONADO NA LISTA:  26776645850
Total de CPFs:  9
----
CPF ADICIONADO NA LISTA:  12585665836
Total de CPFs:  10
----
CPF ADICIONADO NA LISTA:  08794050740
Total de CPFs:  11
----
CPF ADICIONADO NA LISTA:  02599122716
Total de CPFs:  12
----
CPF ADICIONADO NA LISTA:  85798594505
Total de CPFs:  13
----
CPF ADICIONADO NA LISTA:  01400591058
Total de CPFs:  14
----
CPF ADICIONADO NA LISTA:  74729462815
Total de CPFs:  15
----
CPF ADICIONADO NA LISTA:  57661260859
Total de CPFs:  16
----
CPF ADICIONADO NA

KeyboardInterrupt: 

In [21]:
#BOA VISTA - conectando a api, capturando o retorno e salvando no nosso banco
url_api_boa_vista = "https://www.scpc.inf.br/cgi-bin/spcaxml"
header = { 'Accept': 'application/text' }

cpfs = ['57609098949', '83805290900', '14340587869']
for cpf in cpfs:
    body = "<SPCA-XML xmlns:xsi='http://www.w3.org/2001/XMLSchema-instance' xsi:noNamespaceSchemaLocation='https://www.scpc.inf.br/spcn/spcaxmldef.xsd'>  	 	<VERSAO>20190710</VERSAO>      	<SOLICITACAO>         		<S-REGIONAL>0</S-REGIONAL>         		<S-CODIGO>5000</S-CODIGO>         		<S-SENHA>439524</S-SENHA>         		<S-CONSULTA>310</S-CONSULTA>         		<S-SOLICITANTE>FULANO DE TAL</S-SOLICITANTE>         		<S-CPF>"+cpf+"</S-CPF>  		<S-BANCO>1</S-BANCO> 		<S-AGENCIA>1</S-AGENCIA> 		<S-CONTA-CORRENTE>1</S-CONTA-CORRENTE> 		<S-CHEQUE>1</S-CHEQUE> 		<S-CHEQUE-QTDE>1</S-CHEQUE-QTDE> 		<S-CHEQUE-DIGITO>9</S-CHEQUE-DIGITO> 		<S-DDD-1>99</S-DDD-1>         		<S-TELEFONE-1>999999999</S-TELEFONE-1>         		<S-CEP>57480000</S-CEP>         		<S-ACERTA-PRODUTO>             			<S-ACERTA-SCORE>S</S-ACERTA-SCORE> 			<S-ACERTA-SCORE-12>N</S-ACERTA-SCORE-12> 			<S-ACERTA-SCORE-CC>N</S-ACERTA-SCORE-CC> 			<S-ACERTA-SCORE-63>S</S-ACERTA-SCORE-63>   			<S-ACERTA-CHEQUE>S</S-ACERTA-CHEQUE>     		</S-ACERTA-PRODUTO>  	 	</SOLICITACAO>  </SPCA-XML>"
    response_boa_vista = requests.get(url_api_boa_vista, headers=header, data=body)
    table2.put_item(
       Item={
            'document': cpf,
            'bureau': 'boa_vista',
            'data': response_boa_vista.text
            }
    )

In [3]:
url_api_boa_vista = "https://www.scpc.inf.br/cgi-bin/spcaxml"
header = { 'Accept': 'application/text' }
cpf = '06220090446'
body = "<SPCA-XML xmlns:xsi='http://www.w3.org/2001/XMLSchema-instance' xsi:noNamespaceSchemaLocation='https://www.scpc.inf.br/spcn/spcaxmldef.xsd'>  	 	<VERSAO>20190710</VERSAO>      	<SOLICITACAO>         		<S-REGIONAL>0</S-REGIONAL>         		<S-CODIGO>5000</S-CODIGO>         		<S-SENHA>439524</S-SENHA>         		<S-CONSULTA>309</S-CONSULTA>         		<S-SOLICITANTE>FULANO DE TAL</S-SOLICITANTE>         		<S-CPF>"+cpf+"</S-CPF>  		<S-BANCO>1</S-BANCO> 		<S-AGENCIA>1</S-AGENCIA> 		<S-CONTA-CORRENTE>1</S-CONTA-CORRENTE> 		<S-CHEQUE>1</S-CHEQUE> 		<S-CHEQUE-QTDE>1</S-CHEQUE-QTDE> 		<S-CHEQUE-DIGITO>9</S-CHEQUE-DIGITO> 		<S-DDD-1>99</S-DDD-1>         		<S-TELEFONE-1>999999999</S-TELEFONE-1>         		<S-CEP>57480000</S-CEP>         		<S-ACERTA-PRODUTO>             			<S-ACERTA-SCORE>S</S-ACERTA-SCORE> 			<S-ACERTA-SCORE-12>N</S-ACERTA-SCORE-12> 			<S-ACERTA-SCORE-CC>N</S-ACERTA-SCORE-CC> 			<S-ACERTA-SCORE-63>S</S-ACERTA-SCORE-63>   			<S-ACERTA-CHEQUE>S</S-ACERTA-CHEQUE>     		</S-ACERTA-PRODUTO>  	 	</SOLICITACAO>  </SPCA-XML>"
response_boa_vista = requests.get(url_api_boa_vista, headers=header, data=body)
response_boa_vista.text
    

'<?xml version="1.0" encoding="ISO-8859-1"?>\n<SPCA-XML xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="https://www.scpc.inf.br/spcn/spcaxml.xsd">\n    <SOLICITACAO>\n        <S-CODIGO>5000</S-CODIGO>\n        <S-SENHA>439524</S-SENHA>\n        <S-ASSOCIADO/>\n        <S-CONSULTA>309</S-CONSULTA>\n        <S-SOLICITANTE>FULANO DE TAL</S-SOLICITANTE>\n        <S-CPF>6220090446</S-CPF>\n        <S-BANCO>1</S-BANCO>\n        <S-AGENCIA>1</S-AGENCIA>\n        <S-CONTA-CORRENTE>1</S-CONTA-CORRENTE>\n        <S-CHEQUE>1</S-CHEQUE>\n        <S-CHEQUE-DIGITO>9</S-CHEQUE-DIGITO>\n        <S-CHEQUE-QTDE>1</S-CHEQUE-QTDE>\n        <S-DDD-1>99</S-DDD-1>\n        <S-TELEFONE-1>999999999</S-TELEFONE-1>\n        <S-ACERTA-PRODUTO>\n            <S-ACERTA-SCORE>S</S-ACERTA-SCORE>\n            <S-ACERTA-SCORE-06/>\n            <S-ACERTA-SCORE-63>S</S-ACERTA-SCORE-63>\n            <S-ACERTA-SCORE-73/>\n            <S-ACERTA-CHEQUE>S</S-ACERTA-CHEQUE>\n        </S-ACER

In [22]:
#QUOD - conectando a api, capturando o retorno e salvando no nosso banco
headers = {'Content-Type': 'application/json; charset=UTF-8'}
endpoint = 'https://api.empresas.quod.com.br/WsQuodAPI/QuodReport?ver_=1.9'
#cpfs_teste = ['36946118430','77095286272', '28092920001', '34410802542', '66433294006', '22797845722', '47139866074', '93471132040', '37152584206', '23421462992', '32978628405']

for cpf in cpfs:
    
    request_body = {
        "QuodReportRequest": {
            "User": {
            "IndirectClientId": ""
        },
        "Options": {
            "IncludeBestInfo": 1,
            "IncludeCreditRiskIndicators": 1,
            "IncludeCreditRiskData": 1,
            "IncludeQuodScore": 1,
            "IncludeCreditLinesData": 0,
            "IncludeLawSuitData": 1,
            "IncludePartnershipData": 1,
            "IncludeInterpretaData": 1,
            "IncludeCcfData": 1
        },
            "SearchBy": {
                "CPF": cpf
            }
        }
    }

    response_quod = requests.post(endpoint, headers=headers, data=json.dumps(request_body), auth=('07517328000129@esp', '1w4rYh5b'))
    table2.put_item(
        Item={
            'document': cpf,
            'bureau': 'quod',
            'data': response_quod.json()
            }
    )

In [1]:
print('Script finalizado com sucesso!')
print('CPFs adicionados: ', len(cpfs))

Script finalizado com sucesso!


NameError: name 'cpfs' is not defined